In [ ]:
# Import function to calclulate geometry matrix of system
from Impedance_matrix import Matrix, Mnm, L_orthogonal, L_parallel
from Ring_Class import Ring

from Parameters_anisotropic import Rectangle_packing as RP_anizo
from Parameters_anisotropic import Hexagonal_packing as HP_anizo
from Parameters_anisotropic import Params as Params_anizo

from Verifying_MRI.Parameters_MRI import Rectangle_packing as RP_izo
from Verifying_MRI.Parameters_MRI import Rectangle_packing as HP_izo
from Verifying_MRI.Parameters_MRI import Params as Params_izo

In [ ]:
# Calculating currents in each ring 
Grads = {
    'Zero': [[0, 0, 0]],
    'Z': [[0, 0, 0.3], [0, 0, 0.01],         [0, 0, 0.03],         [0, 0, 0.1]],
    'X': [#[0.01, 0, 0],
        [0.03, 0, 0],
        [0.1, 0, 0],
        [0.3, 0, 0]],
}
Omega =  np.linspace(0.9 /sqrt(L*C), 1.4/sqrt(L * C), 500)
n = 20

for type in Grads:
    break
    for grad in Grads[type]:
        print(grad, type)
        Data = solvesystem(n, Omega, grad = grad)
        with open(f"DATA/Data{type}{max(grad)}-n={n}-3.json", "w") as res:
            res.write(json.dumps(Data))
        print(f'Done: {type}-{grad}')
        



In [ ]:
# Drawing GIFs
from PIL import Image
import numpy as np
from numpy import sqrt
Grads = {
    'Zero': [[0, 0, 0]],
    'Z': [[0, 0, 0.01],
        [0, 0, 0.03],
        [0, 0, 0.1],
        [0, 0, 0.3]],
    'X': [[0.01, 0, 0],
        [0.03, 0, 0],
        [0.1, 0, 0],
        [0.3, 0, 0]],
}
Omega =  np.linspace(0.9 /sqrt(L*C), 1.4/sqrt(L * C), 500)
for type in Grads:
    for orientation in ['ZX', 'YZ', 'XY']:
        for grad in Grads[type]:
            frames = []
            for omega in Omega:
                frame = Image.open(f'/Users/shuramakarenko/Discrete_Model_Of_Metamaterial_Lens/Code/Plots/{type}-grad = {max(grad)}/{orientation}3-f={round(omega/2/pi/10 ** 6, 2)}MHz.png')
                frames.append(frame)
            frames[0].save(f'/Users/shuramakarenko/Discrete_Model_Of_Metamaterial_Lens/Code/Plots/GIFS/CD-{orientation}-20x20x20Rings3 grad{type} ={max(grad)}.gif',
                           save_all = True,
                           append_images = frames[1:], 
                           optimize = True, 
                           duration = 150, 
                           loop = 1)

In [ ]:
# Work with Data and calculate polarisation
import matplotlib.pyplot as plt
import json
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

Polarisation = {}
MaxCurrents = {}
MinCurrents = {}
FullGradPolarisation = {}
FullGradMaxCurrents = {}
FullGradMinCurrents = {}



Grads = {
    'Zero': [[0, 0, 0]],
    'Z': [[0, 0, 0.01],
        [0, 0, 0.03],
        [0, 0, 0.1],
        [0, 0, 0.3]],
    'X': [[0.01, 0, 0],
        [0.03, 0, 0],
        [0.1, 0, 0],
        [0.3, 0, 0]],
}
for type in Grads:
    for grad in Grads[type]:
        with open(f"DATA/Data{type}{max(grad)}-n=20.json", "r") as res:
            Data = json.loads(res.read())
        

        N = Data['N']
        Omega = np.array(Data['Omega'])
        a, a1, b, b1, c, c1, w, L, C, R = Data['params']
        M = np.array(Data['Matrix'])
        Number = len(M)
        Currents = np.array(Data['RealCurrents'], dtype = complex) + 1j * np.array(Data['ImagCurrents'], dtype = complex)
        BorderCurrents = np.array(Data['BorderCurrentReal'], dtype = complex) + 1j * np.array(Data['BorderCurrentImag'], dtype = complex)
        if max(grad) != 0:
            Polarisation[f'{type}{grad}'] = Data['RealPolarisation'] + 1j * np.array(Data['ImagPolarisation'], dtype = complex)
            MaxCurrents[f'{type}{grad}'] = Data['RealMaxCurrents'] + 1j * np.array(Data['ImagMaxCurrents'], dtype = complex)
            MinCurrents[f'{type}{grad}'] = Data['RealMinCurrents'] + 1j * np.array(Data['ImagMinCurrents'], dtype = complex)
        if max(grad) == max([max(grad2) for grad2 in Grads[type]]):
            FullGradPolarisation[max(grad)] = Data['BorderRealPolarisation'] + 1j * np.array(Data['BorderImagPolarisation'], dtype = complex)
            FullGradMaxCurrents[max(grad)] = Data['BorderRealMaxCurrents'] + 1j * np.array(Data['BorderImagMaxCurrents'], dtype = complex)
            FullGradMinCurrents[max(grad)] = Data['BorderRealMinCurrents'] + 1j * np.array(Data['BorderImagMinCurrents'], dtype = complex)
        if max(grad) == 0:
            FullGradPolarisation[0] = Data['RealPolarisation'] + 1j * np.array(Data['ImagPolarisation'], dtype = complex)
            FullGradMaxCurrents[0] = Data['RealMaxCurrents'] + 1j * np.array(Data['ImagMaxCurrents'], dtype = complex)
            FullGradMinCurrents[0] = Data['RealMinCurrents'] + 1j * np.array(Data['ImagMinCurrents'], dtype = complex)
        print(f'Data: got {type}-grad = {grad}')

        
        Currents3D = Currents.reshape((len(Omega), N['z'], N['y'], N['z']))
        MaxMiddleXZCurrents = np.real(Currents3D[:, :, N['y']//2, :]).max()
        MinMiddleXZCurrents = np.real(Currents3D[:, :, N['y']//2, :]).min()
        MaxMiddleYZCurrents = np.real(Currents3D[:, :, :, N['x']//2]).max()
        MinMiddleYZCurrents = np.real(Currents3D[:, :, :, N['x']//2]).min()
        MaxMiddleXYCurrents = np.real(Currents3D[:, N['z']//2, :, :]).max()
        MinMiddleXYCurrents = np.real(Currents3D[:, N['z']//2, :, :]).min()

        for I, omega in zip(Currents3D, Omega):
            
            X = list(range(N['x']))
            Y = list(range(N['y']))
            Z = list(range(N['z']))
            fig, ax = plt.subplots()
            fig.set_size_inches((10, 5))
            CurrentsXZ = I[:,N['y']//2,:]
            LinesXZ = plt.contour(X, Z, CurrentsXZ, 6, colors = 'black')
            Imax, Imin = CurrentsXZ.max(), CurrentsXZ.min()
            DeltaCurrents = np.real(Imax) - np.real(Imin)
            MaxDeltaCurrents = MaxMiddleXZCurrents - MinMiddleXZCurrents
            amplitude = DeltaCurrents/MaxDeltaCurrents
            middlepos = (Imin-MinMiddleXZCurrents)/(MaxDeltaCurrents - DeltaCurrents)
            ticksnumber = round(amplitude * 10) + 1
            ticks = list(map(lambda x: round(np.real(x), 2), np.linspace(Imax, Imin, ticksnumber)))

            plt.xlabel(r'$n_x$', fontsize = 15)
            plt.ylabel(r'$n_z$', fontsize = 15)
            plt.title(f"$f =${round(omega/2/pi/10 ** 6, 2)},MHz", fontsize = 15)

            plt.contourf(X, Z, CurrentsXZ, 100, cmap=plt.cm.hot)
            plt.clabel(LinesXZ, inline=1, fontsize=15)
            plt.colorbar(label = 'Current', fraction = 0.09, pad = 0.25, 
                         shrink = 10, aspect = amplitude/0.09, anchor = (middlepos, 0),
                         ticks = ticks, orientation = 'horizontal'
                         ).ax.tick_params(rotation = 45)
            plt.savefig(f'/Users/shuramakarenko/Discrete_Model_Of_Metamaterial_Lens/Code/Plots/{type}-grad = {max(grad)}/ZX3-f={round(omega/2/pi/10 ** 6, 2)}MHz.png')
            
            # X = list(range(N['x']))
            # Y = list(range(N['y']))
            # Z = list(range(N['z']))
            # fig, ax = plt.subplots()
            # fig.set_size_inches((10, 5))
            # CurrentsYZ = I[:,:,N['x']//2]
            # LinesYZ = plt.contour(X, Z, CurrentsYZ, 6, colors = 'black')
            # Imax, Imin = CurrentsYZ.max(), CurrentsYZ.min()
            # DeltaCurrents = np.real(Imax) - np.real(Imin)
            # MaxDeltaCurrents = MaxMiddleYZCurrents - MinMiddleYZCurrents
            # amplitude = DeltaCurrents/MaxDeltaCurrents
            # middlepos = (Imin-MinMiddleYZCurrents)/(MaxDeltaCurrents - DeltaCurrents)
            # ticksnumber = round(amplitude * 10) + 1
            # ticks = list(map(lambda x: round(np.real(x), 2), np.linspace(Imax, Imin, ticksnumber)))

            # plt.xlabel(r'$n_y$', fontsize = 15)
            # plt.ylabel(r'$n_z$', fontsize = 15)
            # plt.title(f"$f =${round(omega/2/pi/10 ** 6, 2)},MHz", fontsize = 15)

            # plt.contourf(Y, Z, CurrentsYZ, 100, cmap=plt.cm.hot)
            # plt.clabel(LinesYZ, inline=1, fontsize=15)
            # plt.colorbar(label = 'Current', fraction = 0.09, pad = 0.25, 
            #              shrink = 10, aspect = amplitude/0.09, anchor = (middlepos, 0),
            #              ticks = ticks, orientation = 'horizontal'
            #              ).ax.tick_params(rotation = 45)
            # plt.savefig(f'/Users/shuramakarenko/Discrete_Model_Of_Metamaterial_Lens/Code/Plots/{type}-grad = {max(grad)}/YZ3-f={round(omega/2/pi/10 ** 6, 2)}MHz.png')

            # X = list(range(N['x']))
            # Y = list(range(N['y']))
            # Z = list(range(N['z']))
            # fig, ax = plt.subplots()
            # fig.set_size_inches((7, 7))
            # CurrentsXY = I[N['z']//2, :, :]
            # LinesXY = plt.contour(X, Z, CurrentsXY, 6, colors = 'black')
            # Imax, Imin = CurrentsXY.max(), CurrentsXY.min()
            # DeltaCurrents = np.real(Imax) - np.real(Imin)
            # MaxDeltaCurrents = MaxMiddleXYCurrents - MinMiddleXYCurrents
            # amplitude = DeltaCurrents/MaxDeltaCurrents
            # middlepos = (Imin-MinMiddleXYCurrents)/(MaxDeltaCurrents - DeltaCurrents)
            # ticksnumber = round(amplitude * 10) + 1
            # ticks = list(map(lambda x: round(np.real(x), 2), np.linspace(Imax, Imin, ticksnumber)))

            # plt.xlabel(r'$n_x$', fontsize = 15)
            # plt.ylabel(r'$n_y$', fontsize = 15)
            # plt.title(f"$f =${round(omega/2/pi/10 ** 6, 2)},MHz", fontsize = 15)

            # plt.contourf(X, Y, CurrentsXY, 100, cmap=plt.cm.hot)
            # plt.clabel(LinesXY, inline=1, fontsize=15)
            # plt.colorbar(label = 'Current', fraction = 0.03, pad = 0.05, 
            #              shrink = 10, aspect = amplitude/0.03, anchor = (0, middlepos),
            #              ticks = ticks
            #              )
            # plt.savefig(f'/Users/shuramakarenko/Discrete_Model_Of_Metamaterial_Lens/Code/Plots/{type}-grad = {max(grad)}/XY3-f={round(omega/2/pi/10 ** 6, 2)}MHz.png')





In [ ]:
# Polarisation plots
for i in Polarisation:
    print(i)
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.xlabel(r'$f$, MHz')
plt.ylabel('Polarisation, imag')

plt.plot(Omega/2/pi/10**6, FullGradPolarisation[0], label = f'no grad', linestyle = '--')
plt.plot(Omega/2/pi/10**6, FullGradPolarisation[0.3], label = f'all 30% different', linestyle = '--')
plt.vlines(1/2/pi/10 ** 6 /sqrt(L *C), 0, max(FullGradPolarisation[0.3]), label = 'resonance frequence', linestyle = ':')
# plt.plot(Omega/2/pi/10**6, Polarisation[f'Z[0, 0, 0.1]'], label = f'z-grad = {int(0.1 * 100)}%')
# plt.ylim(0, 3)
# plt.xlim(65, 75)
for grad in Grads['Z']:
    plt.plot(Omega/2/pi/10**6, Polarisation[f'Z{grad}'], label = f'z-grad = {int(max(grad) * 100)}%')

plt.legend()
plt.show()


plt.figure(figsize=(12, 8))
plt.xlabel(r'$f$, MHz')
plt.ylabel('Polarisation, imag')

plt.plot(Omega/2/pi/10**6, FullGradPolarisation[0], label = f'no grad', linestyle = '--')
plt.plot(Omega/2/pi/10**6, FullGradPolarisation[0.3], label = f'all 30% different', linestyle = '--')
plt.vlines(1/2/pi/10 ** 6 /sqrt(L *C), 0, max(FullGradPolarisation[0.3]), label = 'resonance frequence', linestyle = ':')

for grad in Grads['X']:
    plt.plot(Omega/2/pi/10**6, Polarisation[f'X{grad}'], label = f'x-grad = {int(max(grad) * 100)}%')

plt.legend()
plt.show()

plt.figure(figsize=(12, 8))
plt.xlabel(r'$f$, MHz')
plt.ylabel(r'$I_{max}$')

plt.plot(Omega/2/pi/10**6, FullGradMaxCurrents[0], label = f'no grad', linestyle = '--')
plt.plot(Omega/2/pi/10**6, FullGradMaxCurrents[0.3], label = f'all 30% different', linestyle = '--')
plt.vlines(1/2/pi/10 ** 6 /sqrt(L *C), min(FullGradMaxCurrents[0.3]), max(FullGradMaxCurrents[0.3]), label = 'resonance frequence', linestyle = ':')

for grad in Grads['Z']:
    plt.plot(Omega/2/pi/10**6, MaxCurrents[f'Z{grad}'], label = f'z-grad = {int(max(grad) * 100)}%')

plt.legend()
plt.show() 


plt.figure(figsize=(12, 8))
plt.xlabel(r'$f$, MHz')
plt.ylabel(r'$I_{min}$')

plt.plot(Omega/2/pi/10**6, FullGradMinCurrents[0], label = f'no grad', linestyle = '--')
plt.plot(Omega/2/pi/10**6, FullGradMinCurrents[0.3], label = f'all 30% different', linestyle = '--')
plt.vlines(1/2/pi/10 ** 6 /sqrt(L *C), min(FullGradMinCurrents[0.3]), max(FullGradMinCurrents[0.3]), label = 'resonance frequence', linestyle = ':')

for grad in Grads['Z']:
    plt.plot(Omega/2/pi/10**6, MinCurrents[f'Z{grad}'], label = f'z-grad = {int(max(grad) * 100)}%')

plt.legend()
plt.show() 


plt.figure(figsize=(12, 8))
plt.xlabel(r'$f$, MHz')
plt.ylabel(r'$I_{max}$')

plt.plot(Omega/2/pi/10**6, FullGradMaxCurrents[0], label = f'no grad', linestyle = '--')
plt.plot(Omega/2/pi/10**6, FullGradMaxCurrents[0.3], label = f'all 30% different', linestyle = '--')
plt.vlines(1/2/pi/10 ** 6 /sqrt(L *C), min(FullGradMaxCurrents[0.3]), max(FullGradMaxCurrents[0.3]), label = 'resonance frequence', linestyle = ':')
for grad in Grads['X']:
    plt.plot(Omega/2/pi/10**6, MaxCurrents[f'X{grad}'], label = f'x-grad = {int(max(grad) * 100)}%')

plt.legend()
plt.show() 


plt.figure(figsize=(12, 8))
plt.xlabel(r'$f$, MHz')
plt.ylabel(r'$I_{min}$')

plt.plot(Omega/2/pi/10**6, FullGradMinCurrents[0], label = f'no grad', linestyle = '--')
plt.plot(Omega/2/pi/10**6, FullGradMinCurrents[0.3], label = f'all 30% different', linestyle = '--')
plt.vlines(1/2/pi/10 ** 6 /sqrt(L *C), min(FullGradMinCurrents[0.3]), max(FullGradMinCurrents[0.3]), label = 'resonance frequence', linestyle = ':')

for grad in Grads['X']:
    plt.plot(Omega/2/pi/10**6, MinCurrents[f'X{grad}'], label = f'x-grad = {int(max(grad) * 100)}%')

plt.legend()
plt.show() 

plt.figure(figsize=(12, 8))
plt.xlabel(r'$f$, MHz')
plt.ylabel(r'$I$')

plt.plot(Omega/2/pi/10**6, FullGradMaxCurrents[0], label = f'no grad', linestyle = '--')
plt.plot(Omega/2/pi/10**6, FullGradMaxCurrents[0.3], label = f'all 30% different', linestyle = '--')
plt.plot(Omega/2/pi/10**6, MaxCurrents[f'Z[0, 0, 0.3]'], label = f'z-grad = {int(0.3 * 100)}%')
plt.plot(Omega/2/pi/10**6, MaxCurrents[f'X[0.3, 0, 0]'], label = f'x-grad = {int(0.3 * 100)}%')
plt.vlines(1/2/pi/10 ** 6 /sqrt(L *C), min(FullGradMaxCurrents[0.3]), max(FullGradMaxCurrents[0.3]), label = 'resonance frequence', linestyle = ':')

plt.legend()
plt.show() 


plt.figure(figsize=(12, 8))
plt.xlabel(r'$f$, MHz')
plt.ylabel(r'$I$')

plt.plot(Omega/2/pi/10**6, FullGradMinCurrents[0], label = f'no grad', linestyle = '--')
plt.plot(Omega/2/pi/10**6, FullGradMaxCurrents[0.3], label = f'all 30% different', linestyle = '--')
plt.plot(Omega/2/pi/10**6, MinCurrents[f'Z[0, 0, 0.3]'], label = f'z-grad = {int(0.3 * 100)}%')
plt.plot(Omega/2/pi/10**6, MinCurrents[f'X[0.3, 0, 0]'], label = f'x-grad = {int(0.3 * 100)}%')
plt.vlines(1/2/pi/10 ** 6 /sqrt(L *C), min(FullGradMinCurrents[0.3]), max(FullGradMinCurrents[0.3]), label = 'resonance frequence', linestyle = ':')

plt.legend()
plt.show() 


plt.figure(figsize=(12, 8))
plt.xlabel(r'$f$, MHz')
plt.ylabel(r'$I$')

plt.plot(Omega/2/pi/10**6, FullGradPolarisation[0], label = f'no grad', linestyle = '--')
plt.plot(Omega/2/pi/10**6, FullGradPolarisation[0.3], label = f'all 30% different', linestyle = '--')
plt.plot(Omega/2/pi/10**6, Polarisation[f'Z[0, 0, 0.3]'], label = f'z-grad = {int(0.3 * 100)}%')
plt.plot(Omega/2/pi/10**6, Polarisation[f'X[0.3, 0, 0]'], label = f'x-grad = {int(0.3 * 100)}%')
print(max(Polarisation[f'Z[0, 0, 0.3]']), max(Polarisation[f'X[0.3, 0, 0]']) )
plt.vlines(1/2/pi/10 ** 6 /sqrt(L *C), min(FullGradPolarisation[0.3]), max(FullGradPolarisation[0.3]), label = 'resonance frequence', linestyle = ':')

plt.legend()
plt.show() 



# plt.figure(figsize=(12, 8))
# plt.xlabel(r'$f$, MHz')
# plt.ylabel('Polarisation, imag')

# plt.plot(Omega/2/pi/10**6, Polarisation[f'z0'], label = f'no grad', linestyle = '--')
# for grad in (0.3, 0.2, 0.1, 0.05):
#     plt.plot(Omega/2/pi/10**6, Polarisation[f'z{grad}'], label = f'z-grad = {int(grad * 100)}%')

# for grad in [0.3]:
#     plt.plot(Omega/2/pi/10**6, BasePolarisation[f'z{grad}'], label = f'all 30% different', linestyle = '--')
# plt.legend()
# plt.show()   

# plt.figure(figsize=(12, 8))
# plt.xlabel(r'$f$, MHz')
# plt.ylabel('Polarisation, imag')
# plt.plot(Omega/2/pi/10**6, Polarisation[f'z0'], label = f'no grad', linestyle = '--')
# for grad in (0.3, 0.2, 0.1):
#     plt.plot(Omega/2/pi/10**6, Polarisation[f'x{grad}'], label = f'x-grad = {int(grad * 100)}%')

# for grad in [0.3]:
#     plt.plot(Omega/2/pi/10**6, BasePolarisation[f'z{grad}'], label = f'all 30% different', linestyle = '--')
# plt.legend()
# plt.show()  

# plt.figure(figsize=(12, 8))
# plt.xlabel(r'$f$, MHz')
# plt.ylabel('Polarisation, imag')
# plt.plot(Omega/2/pi/10**6, Polarisation[f'z0'], label = f'no grad', linestyle = '--')
# for grad in (0.3, 0.2, 0.1):
#     plt.plot(Omega/2/pi/10**6, Polarisation[f'multy{grad}'], label = f'multy-grad = {int(grad * 100)}%')

# for grad in [0.3]:
#     plt.plot(Omega/2/pi/10**6, BasePolarisation[f'z{grad}'], label = f'all 30% different', linestyle = '--')
# plt.legend()
# plt.show()  

# plt.figure(figsize=(12, 8))
# plt.xlabel(r'$f$, MHz')
# plt.ylabel('Polarisation, imag')
# plt.plot(Omega/2/pi/10**6, Polarisation[f'Zero0'], label = f'no grad', linestyle = '--')
# for grad in ('Z', 'X', 'Multy'):
#     plt.plot(Omega/2/pi/10**6, Polarisation[f'{grad}0.3'], label = f'{grad}-grad = 30%')

# for grad in [0.3]:
#     plt.plot(Omega/2/pi/10**6, FullGradPolarisation[f'Z{grad}'], label = f'all 30% different', linestyle = '--')
# plt.legend()
# plt.show()  